In [ ]:
from fastai import *
from fastai.vision import *
from sklearn.model_selection import KFold

In [ ]:
!ls ../input

In [ ]:
PATH = Path('../input')

In [ ]:
def create_data(valid_idx):
    test = ImageList.from_df(sub_csv, path=PATH/'test', folder='test')
    data = (ImageList.from_df(df, path=PATH/'train', folder='train')
            .split_by_idx(valid_idx)
            .label_from_df()
            .add_test(test)
            .transform(get_transforms(flip_vert=True, max_rotate=25, max_zoom=1.2, max_lighting=0.3))
            .databunch(path='.', bs=64)
            .normalize(imagenet_stats)
           )
    return data    

In [ ]:
df = pd.read_csv(PATH/'train.csv')
df.head()

In [ ]:
sub_csv = pd.read_csv(PATH/'sample_submission.csv')
sub_csv.head()

In [ ]:
kf = KFold(n_splits=5, random_state=5)
epochs = 6

In [ ]:
lr = 1e-2

In [ ]:
preds = []
for train_idx, valid_idx in kf.split(df):
    data = create_data(valid_idx)
    learn = create_cnn(data, models.densenet201, metrics=[accuracy])
    learn.fit_one_cycle(epochs, slice(lr))
    learn.unfreeze()
    learn.fit_one_cycle(epochs, slice(lr/400, lr/4))
    learn.fit_one_cycle(epochs, slice(lr/800, lr/8))
    preds.append(learn.get_preds(ds_type=DatasetType.Test))

In [ ]:
ens = torch.cat([preds[i][0][:,1].view(-1, 1) for i in range(5)], dim=1)

In [ ]:
ens1  = (ens.mean(1)>=0.5).long(); ens1[:10]

In [ ]:
ens2 = (ens.mean(1)>0.5).long(); ens2[:10]

In [ ]:
sub_csv['has_cactus'] = ens1

In [ ]:
sub_csv.to_csv('submission.csv', index=False)

In [ ]:
sub_csv['has_cactus'] = ens2

In [ ]:
sub_csv.to_csv('submission2.csv', index=False)